# Analysis Notebook - Hierarchical Bayesian Modelling

In [16]:
tissue_index <- 17
as_site_type <- 'se'

## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## Prerequisite input files

Before starting the execution of the following code, make sure you have available in the folders `sbas/data` and `sbas/assets` the files listed below as prerequisites.

###  **`sbas/data`**.
The present analysis requires the following files to be present in the folder **`sbas/data`** uncompressed as the filenames indicate below:

- [x] `fromGTF.{as_site_type}.txt`
- [x] `GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct`
- [x] `rmats_final.{as_site_type}.jc.ijc.txt`
- [x] `rmats_final.{as_site_type}.jc.sjc.txt`
- [x] `limma::topTable()` dataframes written as csv, filenames shsould follow the convention: `{as_site_type}_{tissue_name}_AS_model_B_sex_as_events.csv`
- [x] `srr_pdata.csv`: the corrected GTEx data as created by the forked yarn and in the `annes-changes` branch https://github.com/TheJacksonLaboratory/yarn/tree/annes-changes with the SRR data as used in the `rMATS 3.2.5` experiment.

### **`sbas/assets`**
The present analysis requires the following files to be present in the folder **`sbas/assets`**.

- [x] `tissues.tsv`: metadata file with information on which tissues will be used for analysis -- only using the `include` option for analysis
- [x] `splice-relevant-genes.txt`: list of RNA binding proteins that are annotated to splicing relevant functions from GO.


**NOTE**: For convenience there are two `.tar.gz` archives with the contents described above.

```
gs://robinson-bucket/notebooks/bayesian-modeling/data_bayesian_se_AS_model_B_sex_as_events.tar.gz
gs://robinson-bucket/notebooks/bayesian-modeling/assets_bayesian_se_AS_model_B_sex_as_events.tar.gz
```

Before running the notebook, one can unpack the contents of the archives in the `sbas-nf/data` folder as required by executing the following commands:

```bash
# git clone https://github.com/TheJacksonLaboratory/sbas-nf
tar xvzf data_bayesian_{as_site_type}_AS_model_B_sex_as_events.tar.gz -C data/
tar xvzf assets_bayesian_{as_site_type}_AS_model_B_sex_as_events.tar.gz -C assets/

```


## Loading dependencies

If `conda` is available on your environment you can install the required dependencies by running the following commands:


```bash
time conda install -y r-base==3.6.2 &&
conda install -y r-ggplot2 r-ggsci r-coda r-rstan r-rjags r-compute.es r-snakecase &&
Rscript -e 'install.packages("runjags", repos = "https://cloud.r-project.org/")'
```



In [17]:
# Start the clock!
start_time <- Sys.time()

In [18]:
# dataviz dependencies
library(ggplot2)
library(ggsci)
library(grid)
library(gridExtra)
library(stringr)
library(snakecase)

# BDA2E-utilities dependencies
library(rstan)
library(parallel)
library(rjags)
library(runjags)
library(compute.es)

Previously used list of tissues to use for the Hierarchical Bayesian modelling:



```R
tissue.list<-c("Heart - Left Ventricle",
               "Breast - Mammary Tissue",
               "Brain - Cortex.Brain - Frontal Cortex (BA9).Brain - Anterior cingulate cortex (BA24)",
               "Adrenal Gland",
               "Adipose - Subcutaneous",
               "Muscle - Skeletal",
               "Thyroid",
               "Cells - Transformed fibroblasts",
               "Artery - Aorta",
               "Skin - Sun Exposed (Lower leg).Skin - Not Sun Exposed (Suprapubic)")
```

In [19]:
tissues_df <- readr::read_delim("../assets/tissues.tsv", delim = "\t")

Parsed with column specification:
cols(
  name = col_character(),
  female = col_double(),
  male = col_double(),
  include = col_double(),
  display.name = col_character()
)



In [20]:
tissue.list <- tissues_df$name[ tissues_df$include ==1]

In [21]:
message(length(tissue.list), " tissues")
cat(tissue.list, sep = "\n")

39 tissues



adipose_subcutaneous
adipose_visceral_omentum
adrenal_gland
artery_aorta
artery_coronary
artery_tibial
brain_caudate_basal_ganglia
brain_cerebellar_hemisphere
brain_cerebellum
brain_cortex
brain_frontal_cortex_ba_9
brain_hippocampus
brain_hypothalamus
brain_nucleus_accumbens_basal_ganglia
brain_putamen_basal_ganglia
brain_spinal_cord_cervical_c_1
breast_mammary_tissue
cells_cultured_fibroblasts
cells_ebv_transformed_lymphocytes
colon_sigmoid
colon_transverse
esophagus_gastroesophageal_junction
esophagus_mucosa
esophagus_muscularis
heart_atrial_appendage
heart_left_ventricle
liver
lung
muscle_skeletal
nerve_tibial
pancreas
pituitary
skin_not_sun_exposed_suprapubic
skin_sun_exposed_lower_leg
small_intestine_terminal_ileum
spleen
stomach
thyroid
whole_blood


In [22]:
tissue      <- tissue.list[tissue_index]    #can be replaced with a loop or argument to choose a different tissue
as_site_type #can be replaced with a loop or argument to choose a different as_site_type

[1] "se"

In [23]:
tissue
as_site_type

[1] "breast_mammary_tissue"

[1] "se"

## Pattern for choosing `topTable()` files from `limma`

```bash
# {as_site_type} + '_' + {tissue} + '_' + suffix_pattern 
se_skin_not_sun_exposed_suprapubic_AS_model_B_sex_as_events.csv
```

In [24]:
tissue
dataDir <- "../data/"
assetsDir <- "../assets/"
as_site_type
suffix_pattern <- "AS_model_B_sex_as_events.csv"

file.with.de.results <- paste0(dataDir, as_site_type, "_", tissue, "_" , suffix_pattern  )
file.with.de.results
file.exists(file.with.de.results)
system( paste0("ls -l ", file.with.de.results), intern = TRUE )

[1] "breast_mammary_tissue"

[1] "se"

[1] "../data/se_breast_mammary_tissue_AS_model_B_sex_as_events.csv"

[1] TRUE

[1] "-rw-r--r-- 1 jovyan users 4688985 Jun 26 12:05 ../data/se_breast_mammary_tissue_AS_model_B_sex_as_events.csv"

**NOTE**:
The parameters `tissue_index` and `as_site_type` will be passed from papermill and added in the fist chunk of the notebook upon execution. Do not assign to a hardcoded value anywhere in the notebook, as they will ovewrite the assignment in the first chunk.

In [25]:
events.table         <- read.table(file.with.de.results, sep = ",")
head(events.table, 2)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
XIST-10154,-6.796072,1.385720,-38.73465,5.301573e-131,1.523859e-126,280.9110
XIST-10149,-7.124726,1.597306,-38.68238,7.868530e-131,1.523859e-126,280.5225


## Add annotation columns to the topTable dataframe:

The feature information is encoded in the topTable dataframe as rownames. The `ID` and `geneSymbol` variables have been combined in the following pattern:

```console
{geneSymbol}-{ID} 
```

- `ID`: everything **_after_** last occurence of hyphen `-`
example: 
```R
stringr::str_replace("apples - oranges - bananas", "^.+-", "")
```

```console
# output:

' bananas'
```

- `geneSymbol`: everything **_before_** last occurence of `-`
example: 

```R
sub('-[^-]*$', '',"apples - oranges - bananas")
```

```console
# output:

'apples - oranges '
```

```diff
- NOTE: The above solution covers the cases where a hyphen is part of the geneSymbol.
```

In [26]:
cols_initially <- colnames(events.table)
cols_initially

[1] "logFC"     "AveExpr"   "t"         "P.Value"   "adj.P.Val" "B"

In [27]:
events.table[["ID"]] <- stringr::str_replace(rownames(events.table),  "^.+-", "")
events.table[["gene_name"]] <- sub('-[^-]*$', '', rownames(events.table))

In [28]:
keepInOrderCols <- c("gene_name", "ID", cols_initially)

In [29]:
events.table <- events.table[ , keepInOrderCols ]

In [30]:
tail(events.table, 2)

,gene_name,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RPLP0-28659,RPLP0,28659,0.02222658,9.584065,0.1354014,0.8923696,0.9544973,-7.100906
CD74-25493,CD74,25493,-0.01277937,9.839909,-0.1084203,0.9137222,0.9639965,-7.254767


## Define filepaths of required inputs

`file.with.de.results` has been defined above

In [31]:
rbp.table.name        <- paste0(assetsDir, "splice-relevant-genes.txt")
file.exists(rbp.table.name)

[1] TRUE

In [32]:
events.table.name     <- paste0(paste0(paste0(dataDir, "fromGTF."), snakecase::to_upper_lower_case(as_site_type)),".txt")
events.table.name
file.exists(events.table.name)

[1] "../data/fromGTF.SE.txt"

[1] TRUE

In [33]:
inc.counts.file.name  <- paste0(paste0(paste0(dataDir, "rmats_final."), as_site_type),".jc.ijc.txt")
inc.counts.file.name
file.exists(inc.counts.file.name)

[1] "../data/rmats_final.se.jc.ijc.txt"

[1] TRUE

In [34]:
skip.counts.file.name  <- paste0(paste0(paste0(dataDir, "rmats_final."), as_site_type),".jc.sjc.txt")
skip.counts.file.name
file.exists(skip.counts.file.name)

[1] "../data/rmats_final.se.jc.sjc.txt"

[1] TRUE

In [35]:
metadata.file.name    <- paste0(dataDir, "srr_pdata.csv")
file.exists(metadata.file.name)

[1] TRUE

In [36]:
expression.file.name  <- paste0(dataDir, "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct")
file.exists(expression.file.name)

[1] TRUE

## Use the define filepaths to load/read in the tables 

Load the sjc and sjc count matrices, and the list of RNA binding proteins that are annotated to either:
- mRNA splicing, via spliceosome `(GO:0000398)`,
- regulation of mRNA splicing, via spliceosome `(GO:0048024)`, or 
- both. 

The table has the:
- Gene Symbol
- the Uniprot ID (`uprot.id`)
- the NCBI Gene ID (`gene.id`) and 
- boolean columns for being 
  - `S`=mRNA splicing, via spliceosome `(GO:0000398)` and 
  - `R`=regulation of mRNA splicing, via spliceosome `(GO:0048024)`.

### Filtering of the `topTable()` object

- `abs(events.table$logFC)>=log2(1.5)`
- `events.table$adj.P.Val<=0.05`

In [37]:
dim(events.table)
events.table <- events.table[abs(events.table$logFC)>=log2(1.5) & events.table$adj.P.Val<=0.05,]
dim(events.table)
head(events.table,2)

[1] 38733     8

[1] 3163    8

,gene_name,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
XIST-10154,XIST,10154,-6.796072,1.385720,-38.73465,5.301573e-131,1.523859e-126,280.9110
XIST-10149,XIST,10149,-7.124726,1.597306,-38.68238,7.868530e-131,1.523859e-126,280.5225


Make sure this command has been executed before `gunzip sbas/data/fromGTF.*` as the files are expected uncompressed.


In [38]:
annot.table  <- read.table(events.table.name,header=T)
dim(annot.table)
head(annot.table, 1)

[1] 42611    11

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,ENSG00000034152.18,MAP2K3,chr17,+,21287990,21288091,21284709,21284969,21295674,21295769


In [39]:
merged.table <- merge(events.table, annot.table, by="ID")

In [40]:
dim(merged.table)
head(merged.table, 2)

[1] 3163   18

,ID,gene_name,logFC,AveExpr,t,P.Value,adj.P.Val,B,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>
1,10012,RBM4B,0.8450169,1.183428,6.352709,6.335981e-10,4.076604e-08,12.144677,ENSG00000173914.12,RBM4B,chr11,-,66670935,66670983,66668970,66669291,66676667,66677091
2,10013,RBM4B,-0.7621263,1.752806,-4.689350,3.885523e-06,9.641124e-05,3.795546,ENSG00000173914.12,RBM4B,chr11,-,66668614,66669291,66664997,66665578,66676667,66677091


In [41]:
rbp.table    <- read.table(rbp.table.name,sep="\t",header=TRUE)
dim(rbp.table)
head(rbp.table, 1)

[1] 351   6

,Gene,uprot.id,gene.id,S,R,omim
,<fct>,<fct>,<int>,<lgl>,<lgl>,<fct>
1,AAR2,Q9Y312,25980,TRUE,FALSE,n/a


Make sure this command has been executed before `gunzip sbas/data/rmats_final.{as_site_type}.jc.*jc.*` as the files are expected uncompressed.


In [42]:
inc.counts   <- as.data.frame(data.table::fread(inc.counts.file.name))
dim(inc.counts)
inc.counts[1:2,1:3]

[1] 42611  8674

,ID,SRR1068788,SRR1068808
,<int>,<int>,<int>
1,1,0,0
2,2,26,247


In [43]:
skip.counts  <- as.data.frame(data.table::fread(skip.counts.file.name))
dim(skip.counts)
skip.counts[1:2,1:3]

[1] 42611  8674

,ID,SRR1068788,SRR1068808
,<int>,<int>,<int>
1,1,2,0
2,2,0,0


## Check `dim()` of loaded objects

In [44]:
dim(events.table)
dim(annot.table)
dim(merged.table)
dim(rbp.table)
dim(inc.counts)
dim(skip.counts)

[1] 3163    8

[1] 42611    11

[1] 3163   18

[1] 351   6

[1] 42611  8674

[1] 42611  8674

## Read sample info

In [45]:
metadata.file.name
file.exists(metadata.file.name)
system(paste0("ls -l", " ../data/srr_pdata.csv"), intern = TRUE)

[1] "../data/srr_pdata.csv"

[1] TRUE

[1] "-rw-r--r-- 1 jovyan users 4230047 Jun 26 12:07 ../data/srr_pdata.csv"

In [46]:
meta.data <- readr::read_csv(metadata.file.name)
dim(meta.data)
head(meta.data, 1)

Parsed with column specification:
cols(
  .default = col_double(),
  SAMPID = col_character(),
  SMATSSCR = col_character(),
  SMCENTER = col_character(),
  SMPTHNTS = col_character(),
  SMTS = col_character(),
  SMTSD = col_character(),
  SMUBRID = col_character(),
  SMNABTCH = col_character(),
  SMNABTCHT = col_character(),
  SMNABTCHD = col_character(),
  SMGEBTCH = col_character(),
  SMGEBTCHD = col_character(),
  SMGEBTCHT = col_character(),
  SMAFRZE = col_character(),
  SMGTC = col_logical(),
  SMNUMGPS = col_logical(),
  SM550NRM = col_logical(),
  SM350NRM = col_logical(),
  SMMNCPB = col_logical(),
  SMMNCV = col_logical()
  # ... with 6 more columns
)

See spec(...) for full column specifications.



[1] 7872   68

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,SRR
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
GTEX.PW2O.0006.SM.2I3DV,NA,B1,NA,7.4,Blood,Whole Blood,13756,-126,NA,⋯,0.00351302,0.859573,NA,0,50.6829,GTEX-PW2O,1,20-29,0,SRR604002


In [47]:
meta.data$SMTSD[1:3]

[1] "Whole Blood"            "Lung"                   "Adipose - Subcutaneous"

In [48]:
meta.data[["SMTSD"]] <- as.character(meta.data[["SMTSD"]])

In [49]:
meta.data$SMTSD[1:3]

[1] "Whole Blood"            "Lung"                   "Adipose - Subcutaneous"

In [50]:
meta.data <- meta.data[ snakecase::to_snake_case(meta.data$SMTSD) == tissue,]

In [51]:
tissue
dim(meta.data)
meta.data[1:2,1:8]

[1] "breast_mammary_tissue"

[1] 178  68

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
GTEX.S4Q7.1126.SM.4AD6R,0,B1,2 aliquots,6.9,Breast,Breast - Mammary Tissue,8367
GTEX.ZZ64.1226.SM.5E43R,0,B1,2 pieces; fibroadipose tissue without ducts,7.2,Breast,Breast - Mammary Tissue,8367


In [52]:
# Undo snakecase of SMTSD
tissue
tissue <- unique(meta.data$SMTSD)
tissue

[1] "breast_mammary_tissue"

[1] "Breast - Mammary Tissue"

In [53]:
dim(inc.counts)
inc.counts   <- inc.counts[,colnames(inc.counts) %in% meta.data$SRR]
dim(inc.counts)

[1] 42611  8674

[1] 42611   178

In [54]:
dim(skip.counts)
skip.counts  <- skip.counts[,colnames(skip.counts) %in% meta.data$SRR]
dim(skip.counts)

[1] 42611  8674

[1] 42611   178

## This notebook only signficant events are used - filtering now on Adj.P Val

If there are more than 100 events that are significant, reduce this by ordering on the adjusted p-value (column Adj.P.Val).  

Then update the ijc and sjc matrices and the merged.table (annotations)


In [55]:
if (nrow(skip.counts) > 100 ) {
    head(merged.table,2)
    dim(merged.table)
    order_by_adj.P.Val <- order(merged.table$adj.P.Val,decreasing=TRUE)
    head(merged.table[order_by_adj.P.Val,],2)
    keep_IDs <- merged.table[order_by_adj.P.Val,]$ID[1:100]
    head(keep_IDs)
    keep_IDs <- keep_IDs[order(keep_IDs, decreasing=FALSE)]
    head(keep_IDs)
    # counts table will be identical in rows
    counts.keep <- as.character(rownames(skip.counts)) %in% as.character(keep_IDs)
    table(counts.keep)
    # not so sure about merged.table - so lets have a different logical key
    merged.keep <- as.character(merged.table$ID) %in% as.character(keep_IDs)
    table(merged.keep)
    
    # update the sjc counts 
    dim(skip.counts)
    skip.counts  <- skip.counts[counts.keep,]
    dim(skip.counts)
 
    # update the ijc counts
    dim(inc.counts)
    inc.counts   <- inc.counts[counts.keep,]
    dim(inc.counts)
    
    # merged table
    dim(merged.table)
    merged.table       <- merged.table[merged.keep,] 
    dim(merged.table)
}

[1] 100  18

In [56]:
dim(inc.counts)
dim(skip.counts)
dim(merged.table)

[1] 100 178

[1] 100 178

[1] 100  18

## Save R objects

In [57]:
save.image(file = "notebook.RData")
file.exists("notebook.RData")
system("pwd && ls -l notebook.RData", intern = TRUE)

[1] TRUE

[1] "/mnt/shared/gcp-user/session_data/sbas/sbas-nf/papermill-notebooks"
[2] "-rw-r--r-- 1 jovyan users 1483704 Jul  8 00:27 notebook.RData"

## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **"artefacts"**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [58]:
figure_id       <- "bayesian-modeling"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data/ && find . -type f -exec sha256sum {} \\; > ../metadata/",  figure_id, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

Generating sha256 checksums of the artefacts in the `..data/` directory .. 



character(0)

Done!




sha256sum,file
<chr>,<chr>
ebf034c4717382745507dc9c06940cd04e3ef201f10683df321b851913e4dce5,./se_heart_left_ventricle_AS_model_B_sex_as_events.csv
1c54995a1f326c3a253acb65930955db6420e31381d65a03c382453021001082,./se_adipose_visceral_omentum_AS_model_B_sex_as_events.csv
617428171c801be461b6d3cc3861cd81b25fb080495dd48e1c396926fa6b6d84,./se_brain_hippocampus_AS_model_B_sex_as_events.csv
1b3e611864b8510866bb859190f2e00582c6997c6bf384b9f4fbbc8ef64de24a,./se_heart_atrial_appendage_AS_model_B_sex_as_events.csv
ad91ab7a7041c2f701de4164d64f67cd33d247c5d071cea72be8b0c80acada6e,./se_brain_nucleus_accumbens_basal_ganglia_AS_model_B_sex_as_events.csv
c09fc7f68d4c4220108f63bccb0192cea26fabd25e57b7fae1cca9a2a2037b6d,./se_esophagus_muscularis_AS_model_B_sex_as_events.csv
cbabb0187ae4a60c5e4799eca17c33fb5fcf6df90bd60a428e31e94576da0202,./se_skin_sun_exposed_lower_leg_AS_model_B_sex_as_events.csv
474240fce689c04b44fe7e7b2212cb667374aebc1c273b5ebf23c0016a74854f,./se_adrenal_gland_AS_model_B_sex_as_events.csv
94f7d5c011b416d660c5f1f5ab249a0de0e8fe03dc3a75b59a6ccf3d6d39db09,./se_breast_mammary_tissue_AS_model_B_sex_as_events.csv


### 2. Libraries metadata

In [59]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", figure_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", figure_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]

Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..

Done!


Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..

Done!




 setting  value                       
 version  R version 3.6.1 (2019-07-05)
 os       Ubuntu 18.04.3 LTS          
 system   x86_64, linux-gnu           
 ui       X11                         
 language en_US.UTF-8                 
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2020-07-08                  

,package,ondiskversion,loadedversion,path,loadedpath,attached,is_base,date,source,md5ok,library
,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<fct>
coda,coda,0.19.3,0.19-3,/opt/conda/lib/R/library/coda,/opt/conda/lib/R/library/coda,TRUE,FALSE,2019-07-05,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
compute.es,compute.es,0.2.5,0.2-5,/opt/conda/lib/R/library/compute.es,/opt/conda/lib/R/library/compute.es,TRUE,FALSE,2020-04-01,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
ggplot2,ggplot2,3.3.2,3.3.2,/opt/conda/lib/R/library/ggplot2,/opt/conda/lib/R/library/ggplot2,TRUE,FALSE,2020-06-19,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
ggsci,ggsci,2.9,2.9,/opt/conda/lib/R/library/ggsci,/opt/conda/lib/R/library/ggsci,TRUE,FALSE,2018-05-14,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
gridExtra,gridExtra,2.3,2.3,/opt/conda/lib/R/library/gridExtra,/opt/conda/lib/R/library/gridExtra,TRUE,FALSE,2017-09-09,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
rjags,rjags,4.10,4-10,/opt/conda/lib/R/library/rjags,/opt/conda/lib/R/library/rjags,TRUE,FALSE,2019-11-06,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
rstan,rstan,2.19.3,2.19.3,/opt/conda/lib/R/library/rstan,/opt/conda/lib/R/library/rstan,TRUE,FALSE,2020-02-11,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
runjags,runjags,2.0.4.6,2.0.4-6,/opt/conda/lib/R/library/runjags,/opt/conda/lib/R/library/runjags,TRUE,FALSE,2019-12-17,CRAN (R 3.6.1),NA,/opt/conda/lib/R/library
snakecase,snakecase,0.11.0,0.11.0,/opt/conda/lib/R/library/snakecase,/opt/conda/lib/R/library/snakecase,TRUE,FALSE,2019-05-25,CRAN (R 3.6.3),NA,/opt/conda/lib/R/library
